In [1]:
import time
import clr

Hello


In [4]:
clr.AddReference(r"C:\Program Files\Thorlabs\Kinesis\Thorlabs.MotionControl.DeviceManagerCLI.dll")
clr.AddReference(r"C:\Program Files\Thorlabs\Kinesis\Thorlabs.MotionControl.GenericMotorCLI.dll")
clr.AddReference(r"C:\Program Files\Thorlabs\Kinesis\Thorlabs.MotionControl.KCube.DCServoCLI.dll")

clr.AddReference(r"C:\Program Files\Thorlabs\Kinesis\Thorlabs.MotionControl.TCube.DCServoCLI.dll")

from Thorlabs.MotionControl.DeviceManagerCLI import *
from Thorlabs.MotionControl.GenericMotorCLI import *
from Thorlabs.MotionControl.GenericMotorCLI import KCubeMotor
from Thorlabs.MotionControl.GenericMotorCLI.ControlParameters import JogParametersBase
from Thorlabs.MotionControl.KCube.DCServoCLI import *
from System import Decimal

from Thorlabs.MotionControl.TCube.DCServoCLI import *

In [5]:
# Device puerto 6
serial_num = str('83815214')

In [6]:
# Device Puerto 5
serial_num = str('83814949')

In [7]:
DeviceManagerCLI.BuildDeviceList()
controller = TCubeDCServo.CreateTCubeDCServo(serial_num)
# controller = KCubeDCServo.CreateKCubeDCServo(serial_num)

if not controller == None:
    controller.Connect(serial_num)
    
    if not controller.IsSettingsInitialized():
        controller.WaitForSettingsInitialized(3000)
        
    controller.StartPolling(50)
    time.sleep(.1)
    controller.EnableDevice()
    time.sleep(.1)
    
    config = controller.LoadMotorConfiguration(serial_num, DeviceConfiguration.DeviceSettingsUseOptionType.UseFileSettings)
    config.DeviceSettingsName = str('Z725B')
    config.UpdateCurrentConfiguration()
    controller.SetSettings(controller.MotorDeviceSettings, True, False)
    
    print('Homing Motor')
    #Hacemos que el tornillo se mueva hasta su posición base
    controller.Home(60000)
    
    jog_params = controller.GetJogParams()
    jog_params.StepSize = Decimal(1) #Cambiamos cuanto se mueve nuestro tornillo en cada paso 
    jog_params.MaxVelocity = Decimal(1) #Cambiamos la velocidad máxima a la que se mueve en cada paso
    jog_params.JogMode = JogParametersBase.JogModes.SingleStep 
    
    controller.SetJogParams(jog_params) #Incorporamos los nuevos parámetros a las instrucciones de movimiento
    
    pause_time = 2 #Tiempo de pausa entre movimientos (en segundos)
    forward_movements = 4 #Number of forward movements
    backward_movements = 4 #Number of backward movements
    start_time = time.time() #Iniciamos el origen de tiempos
    
    for _ in range(forward_movements):
        print('Moving Motor Forward at t = ', time.time() - start_time)
        controller.MoveJog(MotorDirection.Forward, 60000)
        print('Pause at t = ', time.time() - start_time)
        time.sleep(pause_time)
        
    for _ in range(backward_movements):
        print('Moving Motor Backward at t = ', time.time() - start_time)
        controller.MoveJog(MotorDirection.Backward, 60000)
        print('Pause at t = ', time.time() - start_time)
        time.sleep(pause_time)
    
    
    print('FINISHED at t = ', time.time() - start_time)
    controller.StopPolling()
    controller.Disconnect(False)

Homing Motor
Moving Motor Forward at t =  0.0
Pause at t =  3.8097572326660156
Moving Motor Forward at t =  5.812042474746704
Pause at t =  9.725464344024658
Moving Motor Forward at t =  11.726611137390137
Pause at t =  15.514995336532593
Moving Motor Forward at t =  17.516632080078125
Pause at t =  21.512812852859497
Moving Motor Backward at t =  23.513747453689575
Pause at t =  27.333569526672363
Moving Motor Backward at t =  29.334723711013794
Pause at t =  33.15541124343872
Moving Motor Backward at t =  35.157310009002686
Pause at t =  38.99285173416138
Moving Motor Backward at t =  40.993937730789185
Pause at t =  44.91052746772766
FINISHED at t =  46.91157364845276
